In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Los Angeles County

In [ ]:
la_hispanic_2021 = pd.read_csv('https://raw.githubusercontent.com/jyyang7/reverse_mortgage/main/all_mortgage_data/la_hispanic_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

In [ ]:
rm_la_hispanic_2021 = la_hispanic_2021[(la_hispanic_2021['reverse_mortgage'] == 1)]
rm_la_hispanic_2021

In [ ]:
rm_la_hispanic_2021.to_csv('./reverse_mortgage_data/rm_la_hispanic_2021.csv', index=False)

In [ ]:
#read csv

la_all_2018 = pd.read_csv('./all_mortgage_data/la_all_2018.csv', dtype={
    'county_code': str,
    'census_tract': str
})

la_all_2019 = pd.read_csv('./all_mortgage_data/la_all_2019.csv', dtype={
    'county_code': str,
    'census_tract': str
})

la_all_2020 = pd.read_csv('./all_mortgage_data/la_all_2020.csv', dtype={
    'county_code': str,
    'census_tract': str
})

la_all_2021 = pd.read_csv('./all_mortgage_data/la_all_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

In [ ]:
# filter to reverse mortgage

rm_la_all_2018 = la_all_2018[(la_all_2018['reverse_mortgage'] == 1)]
rm_la_all_2019 = la_all_2019[(la_all_2019['reverse_mortgage'] == 1)]
rm_la_all_2020 = la_all_2020[(la_all_2020['reverse_mortgage'] == 1)]
rm_la_all_2021 = la_all_2021[(la_all_2021['reverse_mortgage'] == 1)]

In [ ]:
rm_la_all_2018.info()

In [ ]:
rm_la_all_2019.info()

In [ ]:
rm_la_all_2020.info()

In [ ]:
rm_la_all_2021.info()

In [ ]:
# concatenate 2018-2021 datasets

rm_la_all = pd.concat((rm_la_all_2018, rm_la_all_2019, rm_la_all_2020, rm_la_all_2021), axis=0)

In [ ]:
rm_la_all.sample(10)

In [ ]:
# export to csv

rm_la_all.to_csv('./reverse_mortgage_data/rm_la_all.csv', index=False)

## Alameda County

In [20]:
# read csv

alameda_all_2018 = pd.read_csv('./all_mortgage_data/alameda_all_2018.csv', dtype={
    'county_code': str,
    'census_tract': str
})

alameda_all_2019 = pd.read_csv('./all_mortgage_data/alameda_all_2019.csv', dtype={
    'county_code': str,
    'census_tract': str
})

alameda_all_2020 = pd.read_csv('./all_mortgage_data/alameda_all_2020.csv', dtype={
    'county_code': str,
    'census_tract': str
})

alameda_all_2021 = pd.read_csv('./all_mortgage_data/alameda_all_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

/var/folders/br/8x3cfw457y1gzxfpj7108n100000gn/T/ipykernel_1600/3133540299.py:3: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,44) have mixed types. Specify dtype option on import or set low_memory=False.
  alameda_all_2018 = pd.read_csv('./all_mortgage_data/alameda_all_2018.csv', dtype={
/var/folders/br/8x3cfw457y1gzxfpj7108n100000gn/T/ipykernel_1600/3133540299.py:8: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,44) have mixed types. Specify dtype option on import or set low_memory=False.
  alameda_all_2019 = pd.read_csv('./all_mortgage_data/alameda_all_2019.csv', dtype={
/var/folders/br/8x3cfw457y1gzxfpj7108n100000gn/T/ipykernel_1600/3133540299.py:13: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  alameda_all_2020 = pd.read_csv('./all_mortgage_data/alameda_all_2020.csv', dtype={
/var/folders/br/8x3cfw457y1gzxfpj7108n100000gn/T/ipykernel_1600/3133540299.py:18: Dtype

In [21]:
# filter to reverse mortgage

alameda_rm_2018 = alameda_all_2018[(alameda_all_2018['reverse_mortgage'] == 1)]
alameda_rm_2019 = alameda_all_2019[(alameda_all_2019['reverse_mortgage'] == 1)]
alameda_rm_2020 = alameda_all_2020[(alameda_all_2020['reverse_mortgage'] == 1)]
alameda_rm_2021 = alameda_all_2021[(alameda_all_2021['reverse_mortgage'] == 1)]

In [26]:
# concatenate 2018-2021 years

alameda_rm_all = pd.concat((alameda_rm_2018, alameda_rm_2019, alameda_rm_2020, alameda_rm_2021), axis=0)
alameda_rm_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3753 entries, 7166 to 144575
Data columns (total 99 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   activity_year                             3753 non-null   int64  
 1   lei                                       3753 non-null   object 
 2   derived_msa-md                            3753 non-null   int64  
 3   state_code                                3753 non-null   object 
 4   county_code                               3753 non-null   object 
 5   census_tract                              3752 non-null   object 
 6   conforming_loan_limit                     3753 non-null   object 
 7   derived_loan_product_type                 3753 non-null   object 
 8   derived_dwelling_category                 3753 non-null   object 
 9   derived_ethnicity                         3753 non-null   object 
 10  derived_race                   

In [27]:
# export
alameda_rm_all.to_csv('./reverse_mortgage_data/alameda_rm_all.csv', index=False)

## Sacramento County

## San Bernardino County